# Programma che effettua lo scraping agli articoli di datajournalism sul sito Nòva del Sole 24 Ore

In [3]:
import time, csv                 # sono importate le diverse librerie per il corretto funzionamento del bot
import pandas as pd
from selenium import webdriver
from datetime import datetime


driver = webdriver.Chrome()   # il driver è associato a Chrome 

driver.get("https://nova.ilsole24ore.com/source/data-journalism-infodata/") # home page del sito di Nova, sezione articoli di datajournalism 

time.sleep(3) # il bot si addormenta per 3 secondi

cookie = driver.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]').click() # viene cliccato il pulsante che accetta i cookie della pagina

time.sleep(2)


altri_articoli = driver.find_element_by_id("load-link")  # id del pulsante "altri contenuti"
for i in range(200):    # 2 per prova ma metterlo su 200 per far uscire circa 2392 articoli ma essendo un sito un po' così può bloccarsi talvolta
    time.sleep(2)
    altri_articoli.click()
    
articoli = driver.find_elements_by_class_name("lead-normal")  # classe degli articoli presenti nella pagina


lista_articoli = []   # un array che conterrà i valori del dizionario art_item1
solo_Link = []        # un array dove saranno inseriti esclusivamente i link degli articoli

for articolo in articoli:
    
    link = articolo.find_element_by_xpath('.//*[@class="title"]/h2/a').get_attribute('href')
    titolo = articolo.find_element_by_xpath('.//*[@class="title"]/h2/a').text # . (seleziona il nodo corrente) poichè senza si ripete il primo
    categoria = articolo.find_element_by_xpath('.//*[@class="title"]/h3/a').text
     
    data_pubblicazione = articolo.find_elements_by_tag_name("time") 
    
    lista_data = []   
    
    for elemento in data_pubblicazione:  # scorre gli elementi della lista data_pubblicazione 
        lista_data.append(elemento.text)       # prende il testo dell'elemento appendendolo all'array lista_data
    


    data = ' '.join(map(str, lista_data)) # map() prende in ingresso una o più liste e una funzione. Per ogni elemento presente in ciascuna lista specificata, viene applicata la funzione str() che lo converte in stringa.
    
    
    
    
    now = datetime.now() # data nel formato Americano e orario corrente
    timestamp = now.strftime("%d/%m/%Y, %H:%M:%S") # Il metodo strftime () restituisce una stringa che rappresenta la data e l'ora utilizzando l'oggetto date, time o datetime
    
    art_item1 = {        # primo dizionario che contiene per ogni articolo: link, titolo, categoria, datina( la data di pubblicazione), timestamp dello scraping
        'Link': link,
        'Titolo': titolo,
        'Categoria': categoria,
        'Data_pubblicazione': data, 
        'Timestamp dello scraping': timestamp
    }
    lista_articoli.append(art_item1)     # il primo dizionario è appeso a lista_articoli
    solo_Link.append(link)               # ogni link è appeso all'array solo_Link

driver.quit()                       # il bot si chiude
df1 = pd.DataFrame(lista_articoli)  # tramite la libreria Pandas, la funzione DataFrame() trasforma la lista corpi in una tabella di oggetti eterogenei



corpi = []                # array che conterrà i "corpi" degli articoli

for indirizzo in solo_Link:    # scorre ogni link della lista solo_Link
    driver = webdriver.Chrome()
     
    driver.get(indirizzo)      # il bot accede all'indirizzo
    
    elementip = driver.find_elements_by_tag_name("p") #lista intera di tutti gli elementi p

    paragrafi = []        # andrà a contenere l'insieme dei paragrafi che comporranno un singolo articolo
    
    for paragrafo in elementip:                # scorre gli elementi della lista elementip
        paragrafi.append(paragrafo.text)       # prende il testo dell'elemento appendendolo all'array paragrafi
        
   
    testo = ' '.join(map(str, paragrafi)) # map() prende in ingresso una o più liste e una funzione. Per ogni elemento presente in ciascuna lista specificata, viene applicata la funzione str() che lo converte in stringa.


    art_item2 = {                         # secondo dizionario che contiene il testo di un singolo articolo
        
            'Testo': testo
    }
    
    corpi.append(art_item2)  # il secondo dizionario è appeso alla lista corpi

    driver.quit()            # il bot si arresta
df2 = pd.DataFrame(corpi)    #tramite la libreria Pandas, la funzione DataFrame() trasforma la lista corpi in una tabella di oggetti eterogenei

dfTotale = pd.concat([df1,df2], axis = 1) # i due dataframe sono concatenati in un unico dataframe
dfTotale.to_csv(r'C:\Users\robeo\Desktop\dataframe_Nòva.csv', index = True, header = True) # il dataframe complessivo è convertito nel formato CSV
    

    
    
